In [ ]:
from positional_encodings.torch_encodings import PositionalEncoding1D, PositionalEncoding2D, PositionalEncoding3D, Summer
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.io import savemat

p_enc_2d = PositionalEncoding2D(16)
y = torch.zeros((1,257,257,16))
Z = p_enc_2d(y)
print(Z.shape) 

PosEncoding = np.zeros((34,257,257))
for i in range(16):
  PosEncoding[i,:,:] = Z[0,:,:,i]
  

e = np.mgrid[-1:1 + (1/128):(1/128),-1:1 + (1/128):(1/128)]
y = e[0,:,:]
x = e[1,:,:]
print(y[127:130,127:130])
print(x[127:130,127:130])
y2 = 1 - y**2 
x2 = 1 - x**2 
PosEncoding[16,:,:] = x
PosEncoding[17,:,:] = y
PosEncoding[18,:,:] = x2
PosEncoding[19,:,:] = y2

for n in range(1,9):
  PosEncoding[19 + n,:,:] = np.cos((n*x)**2+(n*y)**2)

for n in range(1,4):
  PosEncoding[27 + n,:,:] = 1-((0.1*(n)*x)**2+(0.1*(4-n)*y)**2)

PosEncoding[31,:,:] = 0.2/((0.1+(x)**2)+(0.1+(y)**2))

PosEncoding[32,:,:] = 1-np.abs(x)

PosEncoding[33,:,:] = 1-np.abs(y)

# for i in range(34):
#     plt.imshow(PosEncoding[i,:,:])
#     plt.show()

_, axs = plt.subplots(5, 7, figsize=(10, 7.1))

# axs = axs.flatten()
for j in range(5):
    for i in range(7):
      if j*7 + i < 34:
        axs[j,i].imshow(PosEncoding[j*7+i,:,:])
      axs[j,i].axis('off')
# plt.tight_layout()
plt.tight_layout(pad=0.4, w_pad=0.4, h_pad=0.4)
plt.show()

mdic = {"PositionalEncoding": PosEncoding}
savemat("PosEncode.mat", mdic)
## TODO: this cell should be saves seperatly and the result could be loaded at the initially


## Load Postional Condition, Repeat it based on batch_size and Convert it Torch.Tensor
# PosEncoding = scipy.loatmat()
PosEncoding = torch.tensor(PosEncoding)
# PosEncoding = PosEncoding[np.newaxis,:,:,:].repeat(batch_size,1,1,1)
print(PosEncoding.shape)


In [ ]:
import matplotlib.pyplot as plt
import scipy.fftpack
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils
import torchvision
import torch
import glob
from medpy.io import load, save
import sys
from scipy.io import savemat

save_to_dir = 'J:/Anvari/MsThesis/K-Space-Net3/T1_model/'
LINE_CLEAR = '\x1b[2K' # <-- ANSI sequence
train_root = 'J:\\Anvari\\proccessed_data_257\\train\\'
validation_root = 'J:\\Anvari\\proccessed_data_257\\validation\\'
transform_2015 = transforms.Compose([transforms.ConvertImageDtype(torch.float),
                                     transforms.Normalize(0.5, 0.5, 0.5)])
def load_mask(mask_name):
    dir = 'C:\\Users\\mehrdad\\MS_projects\\mask\\Build mask'
    matfile = scipy.io.loadmat(dir + '\\{}.mat'.format(mask_name)) 
    print(type(matfile))
    print(matfile.keys())
    # print(matfile.get("__header__"))
    mask = np.zeros((6,257,257))
    mask_P = matfile.get('PrimaryMask')
    mask_S = matfile.get('SecondaryMask')
    mask_P = np.logical_or(mask_P,np.flip(mask_P))
    mask_S = np.logical_or(mask_S,np.flip(mask_S))
    mask[0,:,:]=mask_S
    mask[1,:,:]=mask_P
    mask[2,:,:]=mask_S
    mask[3,:,:]=mask_P
    mask[4,:,:]=mask_S
    mask[5,:,:]=mask_P
    # print(mask.sum() / (256 * 256))
    print(mask.shape)
    print(mask.dtype)
    print(type(mask))
    plt.imshow(mask_P)
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    plt.imshow(mask_S)
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    plt.imshow(np.logical_or(mask_P,mask_S))
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    return mask

mask = load_mask('GaussianDistribution1DMask_20_257')
mask_c = torch.tensor(1 - mask)

def cal_fft_for_all_channels(x):
    fft = np.zeros_like(x).astype('complex128')
    for i in range(6):
        fft[i, :, :] = scipy.fftpack.fft2(x[i, :, :])
        fft[i, :, :] = scipy.fftpack.fftshift(fft[i, :, :])
    return np.real(fft).astype('float32'),np.imag(fft).astype('float32')

def to_bad_img(x, mask):
    x = (x + 1.) / 2.
    
#     print(f"mask shape is:{x.shape}")
    for i in range(6):
        fft = scipy.fftpack.fft2(x[i, :, :])
        fft = scipy.fftpack.fftshift(fft)
        fft = fft * mask[i,:,:]
        fft = scipy.fftpack.ifftshift(fft)
        x[i, :, :] = scipy.fftpack.ifft2(fft)
        x[i, :, :] = np.abs(x[i, :, :])
        x[i, :, :] = x[i, :, :] * 2 - 1
    return x

class Brats2013_2D(Dataset):
    def __init__(self, root, transform=transform_2015 , PE = PosEncoding):
        self.img_dir = root
        self.positionalEncoding = PE
        self.transform = transform
        file_list = glob.glob(root + "*.mha")
        self.data = []
        for file_path in file_list:
            layer_name = file_path.split("\\")[-1].split(".")[-2].split("_")[-1]
            self.data.append([file_path, int(layer_name)])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, LayerNum = self.data[idx]
        image, image_header = load(img_path)
        image = np.float32(image)
        # image = image[np.newaxis,:,:]
        image_fft_r,image_fft_i = cal_fft_for_all_channels(image)
        image_fft_r = torch.tensor(image_fft_r)
        image_fft_i = torch.tensor(image_fft_i)
        image = torch.tensor(image)
        # print(image.shape)
        ## TODO: should return fft of T1 and T2 and also the layer number
        return image_fft_r, image_fft_i, image, torch.tensor(LayerNum), torch.tensor(self.positionalEncoding)


def imshow(img, batch_size, name):
    # img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
#     print('npimg shape = {}'.format(npimg.shape))
    npimg = np.transpose(npimg, (1, 2, 0))
    # print('npimg shape = {}'.format(npimg.shape))
    plt.figure(figsize=(7, 7))
    plt.imshow(npimg,interpolation = 'none')
    plt.axis('off')
    plt.title('Dataloader Batch-size = {}'.format(batch_size))
    plt.savefig(save_to_dir+'{}.png'.format(name), format='png', dpi=600)
#     plt.show()


def imsave(img,epoch,i,name,batch_size):
    img_slices = np.zeros((6*batch_size,240,240))
    for j in range(batch_size*6):
        img_slices[j,:,:] = img[int(j//6),int(j%6),:,:]
    save(img_slices,save_to_dir+f'{name}_{epoch}_{i}.mha')
    

batch_size = 10
dataset = Brats2013_2D(root=train_root , PE = PosEncoding)
dataset_val = Brats2013_2D(root=validation_root, PE=PosEncoding)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
img_fft_r, img_fft_i, img ,layer , PosEn = next(iter(dataloader))
print(type(img_fft_r))
print(img_fft_r.dtype)
print('img_fft_r shape = {}'.format(img_fft_r.shape))
print('layer = {}'.format(layer.shape))
print('PosEn shape = {}'.format(PosEn.shape))
plt.imshow(img_fft_r[0,0,:,:])
plt.show()

In [ ]:
from torch import nn
import math
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, layer):
        device = layer.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = layer[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


In [ ]:
import torch.nn.functional as F
import torch
from torch import nn
import numpy as np
from torchsummary import summary
from torchvision import models
from einops import rearrange
if torch.cuda.is_available():
    print('it works')
else:
    print('it doesnt work')
device = torch.device("cuda:0" if (torch.cuda.is_available() and 1 > 0) else "cpu")
print('device is {}'.format(device))


class KspaceNetT1(nn.Module):
    def __init__(self,ngpu):
        super(KspaceNetT1, self).__init__()
        self.df_dim = 32     ## TODO: normalizing the input k-space
        self.posEn_dim = 34
        self.layEn_dim = 32
        self.conv1 = nn.Conv2d(10 + self.posEn_dim + self.layEn_dim , 4*self.df_dim, (1, 1), padding=(0, 0))
        self.conv2 = nn.Conv2d(4*self.df_dim, 2*self.df_dim, (1, 1), padding=(0, 0))
        self.conv3 = nn.Conv2d(2*self.df_dim, 1*self.df_dim, (1, 1), padding=(0, 0))
        self.conv4 = nn.Conv2d(1*self.df_dim, 2, (1, 1), padding=(0, 0))

        self.batch1 = nn.BatchNorm2d(4 * self.df_dim)
        self.batch2 = nn.BatchNorm2d(2 * self.df_dim)
        self.batch3 = nn.BatchNorm2d(1 * self.df_dim)

        ## TODO: an MLP model for Layer encoding 
        self.layer_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, self.layEn_dim),
        )
        
        self.layer_mlp2 = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, 2*self.posEn_dim),
        )
        
        self.layer_mlp3 = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, 10),
        )
        
        self.convLEPEforX = nn.Conv2d(self.posEn_dim + 10, 10*2, (1, 1), padding=(0, 0))
        
    def forward(self, x, PE, LE):

        layerEmbeddingLatent = self.layer_mlp(LE)
        
        LEchs = rearrange(layerEmbeddingLatent, "b c -> b c 1 1")
        LEchs = LEchs.repeat(1,1,257,257) ## TODO: check if scale and shifting X using LEchs is any better
        ## TODO: Check if scale and shifting PE using LEch is any better
        LEforPE = self.layer_mlp2(LE)
        LEforPE2 = rearrange(LEforPE, "b c -> b c 1 1")
        scale_shift = LEforPE2.chunk(2, dim=1)
        scale, shift = scale_shift
        PE = PE * (scale + 1) + shift
        ## TODO: Check if scale and shiftinf PE using LEch and concat it to X while the X itself is scaled and shifted by scaled and shifted PE 
        LEforX = self.layer_mlp3(LE)
        LEforX2 = rearrange(LEforX, "b c -> b c 1 1")
        LEforX2 = LEforX2.repeat(1,1,257,257)
        LEPEforX = torch.concat((PE, LEforX2),dim=1)
        LEPEforX2 = self.convLEPEforX(LEPEforX)
        scale_shift = LEPEforX2.chunk(2, dim=1)
        scale, shift = scale_shift
        x = x * (scale + 1) + shift
        ## TODO: Concatinate input (x), Positional Encoding (PE), Layer Encoding (LE)
        
        
        x = torch.concat((x, PE, LEchs),dim=1)

        x = self.conv1(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch1(x)

        x = self.conv2(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch2(x)

        x = self.conv3(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch3(x)

        x = self.conv4(x)
#         x = nn.Tanh()(x)
        
        return x
    

In [ ]:
from tqdm import tqdm
import torch.optim as optim
import torchvision.utils as vutils
from torchmetrics import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import os

ngpu = 1
lr = 0.002
lr_decay = 0.5
decay_every = 5
n_epoch = 5
beta1 = 0.5
batch_size = 10
workers = 1

## check if cuda is available or not?
if torch.cuda.is_available():
    print('CUDA is Available!')
else:
    print('CUDA is NOT Available!')
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print('device is {}'.format(device))

## Empety torch cuda cache
torch.cuda.empty_cache()

## Define the model
model = KspaceNetT1(ngpu).to(device)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

## Parameter initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)




## Define optimizer
print('Setup Adam optimizers for D')
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)

## Define Loss
criterion = torch.nn.MSELoss()

## Define Metrics
psnr = PeakSignalNoiseRatio()
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)

## Load the model if exists, pre-trained model
# if  os.path.isfile(save_to_dir + 'modelt2.pt'):
#     print('Loading trained model ....')
#     torch.load(model, save_to_dir +'modelt2.pt')
#     checkpoint = torch.load(save_to_dir + 'modelt2.pt')
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     model.eval()
    
print(f'scheduler : {scheduler.get_last_lr()}')



## Define some List to Keep track of the Learning 

#  Traing Data
PSNR = []
SSIM = []
MSE_FFT = []
MSE_IMG = []
MSE_RealEven = []
MSE_ImagOdd  = []

#  Validation Data
PSNR_V = []
SSIM_V = []
MSE_FFT_V = []
MSE_IMG_V = []
MSE_RealEven_V = []
MSE_ImagOdd_V  = []
t_p = 1
t_c = 4

mask = torch.tensor(mask).float().to(device)
mask[[3,5],:,:] = 0
mask[4,:,:] = 1
for epoch in range(n_epoch):
    # For each batch in the dataloader
    iters = 0
    for i, data in enumerate(dataloader, 0):
        model.train(True)
        model.zero_grad()

        # load one batch of data
        image_fft_r, image_fft_i, image, LayerNum, PosEncoding = data         
        image_fft_out_unmasked = torch.concat((image_fft_r[:,t_p,:,:][:,np.newaxis,:,:],image_fft_i[:,t_p,:,:][:,np.newaxis,:,:]), dim = 1).to(device)

#         for i in range(6):
#                 image_fft_r[:,i,:,:] = image_fft_r[:,i,:,:] * mask[i,:,:]
#                 image_fft_i[:,i,:,:] = image_fft_i[:,i,:,:] * mask[i,:,:]
#         image_fft_r = image_fft_r.to(device) * mask.to(device)
#         image_fft_i = image_fft_i.to(device) * mask.to(device)
                        
        ## TODO: the resulted imaginary and real channel may not satisfies the fourier properties of real images.
        #        check if defining a loss function that induce such properties can improve the estimation or not?
        ## Seperate Input and Output of the network
        # ListConChIdxs = [0,t_c,2,3,5] ## index 1 is the T1 middle layer that should be predicted
        ListConChIdxs = [0,2,3,t_c,5] ## index 1 is the T1 middle layer that should be predicted
        image_fft_in = torch.concat((image_fft_r[:,ListConChIdxs,:,:],image_fft_i[:,ListConChIdxs,:,:]), dim = 1).to(device)
        image_fft_out = torch.concat((image_fft_r[:,t_p,:,:][:,np.newaxis,:,:],image_fft_i[:,t_p,:,:][:,np.newaxis,:,:]), dim = 1).to(device)
        
      
        PosEncoding = PosEncoding.to(device)
        ## Inference
        Predition = model(image_fft_in.float(),PosEncoding.float(),LayerNum.to(device))
        

#         ## correct values of estimated image using true ones
#         Predition[:,0,:,:] = Predition[:,0,:,:] * mask_c[t_c,:,:].to(device) + image_fft_r[:,t_p,:,:] * mask[t_p,:,:]
#         Predition[:,1,:,:] = Predition[:,1,:,:] * mask_c[t_c,:,:].to(device) + image_fft_i[:,t_p,:,:] * mask[t_p,:,:]

        ## Calculate the Loss term
        Loss_mse = criterion(Predition,image_fft_out_unmasked)
        Loss_real_even = criterion(Predition[:,0,:,:],torch.rot90(Predition[:,0,:,:],2,[1,2]))*10
        Loss_imag_odd = criterion(Predition[:,1,:,:],-1*torch.rot90(Predition[:,1,:,:],2,[1,2]))*10
        complexPrediction  = Predition[:,0,:,:][:,np.newaxis,:,:]+Predition[:,1,:,:][:,np.newaxis,:,:]*1j
        complexPrediction = torch.fft.ifftshift(complexPrediction,dim=(2,3))
        image_Predition = torch.fft.ifft2(complexPrediction)
        image_Predition = torch.real(torch.tensor(image_Predition).float())
        Loss_img = criterion(image_Predition,image[:,t_p,:,:][:,np.newaxis,:,:].to(device)) * 30000
        Loss = Loss_mse + Loss_real_even + Loss_imag_odd + Loss_img
        ## Calculate gradients for G
        Loss.backward()

        ## Optimization Step
        optimizer.step()
        
        ## Print the Loss value after some iteration
        with torch.no_grad():
            if i % 10 == 0:
                psnrValue = psnr(image_Predition.cpu(), image[:,t_p,:,:][:,np.newaxis,:,:].cpu())
                ssimValue = ssim(image_Predition.cpu(), image[:,t_p,:,:][:,np.newaxis,:,:].cpu())
                print(f'Epoch: {epoch}, Iteration: [{len(dataloader)}|{i}], Loss_mse: {Loss_mse:.4f}, Loss_real_even: {Loss_real_even:.4f}, Loss_imag_odd: {Loss_imag_odd:.4f}, Loss_img: {Loss_img:.4f}')
                print(f'Epoch: {epoch}, Iteration: [{len(dataloader)}|{i}], psnr: {psnrValue:.4f}, ssim: {ssimValue:.4f}, max_GT: {torch.max(image[:,t_p,:,:][:,np.newaxis,:,:].cpu()):.4f}, max_Pred: {torch.max(image_Predition.cpu()):.4f},  min_Pred: {torch.min(image_Predition.cpu()):.4f}')
                PSNR.append(psnrValue.item())
                SSIM.append(ssimValue.item())
                MSE_FFT.append(Loss_mse.item())
                MSE_IMG.append(Loss_img.item())
                MSE_RealEven.append(Loss_real_even.item())
                MSE_ImagOdd.append(Loss_imag_odd.item())

        
        ## show the predited image
        model.train(False)
        with torch.no_grad():
            if (i % 100 == 0) or (i == len(dataloader) - 1):
                
                imshow(vutils.make_grid(image_Predition.cpu(), padding=2, nrow=5, normalize=False), batch_size, '{}_{}_fix'.format(epoch, i))
                imshow(vutils.make_grid(image[:,t_p,:,:][:,np.newaxis,:,:].cpu(), padding=2, nrow=5, normalize=False), batch_size, '{}_{}_GroundTruth'.format(epoch, i))
            
                PSNR_Vt = 0
                SSIM_Vt = 0
                MSE_FFT_Vt = 0
                MSE_IMG_Vt = 0
                MSE_RealEven_Vt = 0
                MSE_ImagOdd_Vt  = 0
            
                print('Calculating loss and metrics for Validation dataset ... ')
                for l, data_val in tqdm(enumerate(dataloader_val, 0)):
                    # load one batch of data

                    image_fft_r, image_fft_i, image, LayerNum, PosEncoding = data 
                    
                    image_fft_out_unmasked = torch.concat((image_fft_r[:,t_p,:,:][:,np.newaxis,:,:],image_fft_i[:,t_p,:,:][:,np.newaxis,:,:]), dim = 1).to(device)

#                     for i in range(6):
#                             image_fft_r[:,i,:,:] = image_fft_r[:,i,:,:] * mask[i,:,:].cpu()
#                             image_fft_i[:,i,:,:] = image_fft_i[:,i,:,:] * mask[i,:,:].cpu()

                    # ListConChIdxs = [0,1,2,3,5] ## index 1 is the T1 middle layer that should be predicted
                    ListConChIdxs = [0,2,3,t_c,5] ## index 1 is the T1 middle layer that should be predicted
                    image_fft_in = torch.concat((image_fft_r[:,ListConChIdxs,:,:],image_fft_i[:,ListConChIdxs,:,:]), dim = 1).to(device)
                    image_fft_out = torch.concat((image_fft_r[:,t_p,:,:][:,np.newaxis,:,:],image_fft_i[:,t_p,:,:][:,np.newaxis,:,:]), dim = 1).to(device)

                    PosEncoding = PosEncoding.to(device)
                    ## Inference
                    Predition = model(image_fft_in.float(),PosEncoding.float(),LayerNum.to(device))
                              
                    # correct values of estimated image using true ones
                    # Predition[:,0,:,:] = Predition[:,0,:,:] * mask_c[t_p,:,:] + image_fft_r[:,t_p,:,:].to(device) * mask[t_p,:,:]
                    # Predition[:,1,:,:] = Predition[:,1,:,:] * mask_c[t_p,:,:] + image_fft_i[:,t_p,:,:].to(device) * mask[t_p,:,:]
            
                    ## Calculate the Loss term
                    Loss_mse = criterion(Predition,image_fft_out_unmasked)
                    Loss_real_even = criterion(Predition[:,0,:,:],torch.rot90(Predition[:,0,:,:],2,[1,2])) *10
                    Loss_imag_odd = criterion(Predition[:,1,:,:],-1*torch.rot90(Predition[:,1,:,:],2,[1,2])) *10

                    complexPrediction  = Predition[:,0,:,:][:,np.newaxis,:,:]+Predition[:,1,:,:][:,np.newaxis,:,:]*1j
                    complexPrediction = torch.fft.ifftshift(complexPrediction,dim=(2,3))
                    image_Predition = torch.fft.ifft2(complexPrediction)
                    image_Predition = torch.real(torch.tensor(image_Predition).float())
                    Loss_img = criterion(image_Predition,image[:,t_p,:,:][:,np.newaxis,:,:].to(device)) * 30000

                    psnrValue = psnr(image_Predition.cpu(), image[:,t_p,:,:][:,np.newaxis,:,:].cpu())
                    ssimValue = ssim(image_Predition.cpu(), image[:,t_p,:,:][:,np.newaxis,:,:].cpu())

                    PSNR_Vt = PSNR_Vt + psnrValue.item()
                    SSIM_Vt = SSIM_Vt + ssimValue.item()
                    MSE_FFT_Vt = MSE_FFT_Vt + Loss_mse.item()
                    MSE_IMG_Vt = MSE_IMG_Vt + Loss_img.item()
                    MSE_RealEven_Vt = MSE_RealEven_Vt + Loss_real_even.item()
                    MSE_ImagOdd_Vt  = MSE_ImagOdd_Vt + Loss_imag_odd.item()

                normalizing_factor = len(dataloader_val)

                PSNR_V.append(PSNR_Vt/normalizing_factor)
                SSIM_V.append(SSIM_Vt/normalizing_factor)
                MSE_FFT_V.append(MSE_FFT_Vt/normalizing_factor)
                MSE_IMG_V.append(MSE_IMG_Vt/normalizing_factor)
                MSE_RealEven_V.append(MSE_RealEven_Vt/normalizing_factor)
                MSE_ImagOdd_V.append(MSE_ImagOdd_Vt/normalizing_factor)

                mdic = {"PSNR": PSNR,"SSIM": SSIM,"MSE_FFT": MSE_FFT,"MSE_IMG": MSE_IMG,"MSE_RealEven": MSE_RealEven,"MSE_ImagOdd": MSE_ImagOdd
                       ,"PSNR_V": PSNR_V,"SSIM_V": SSIM_V,"MSE_FFT_V": MSE_FFT_V,"MSE_IMG_V": MSE_IMG_V,"MSE_RealEven_V": MSE_RealEven_V,"MSE_ImagOdd_V": MSE_ImagOdd_V}
                savemat(save_to_dir+"LOG.mat", mdic)
                
                ## to select best model of last epoch for next epoch training
                if len(PSNR_V) > 1:
                    print(f"max:{np.max(PSNR_V[0:-1])},current:{PSNR_Vt/normalizing_factor}")
                    if PSNR_Vt/normalizing_factor > np.max(PSNR_V[0:-1]):
                        print('new best model found!!saving...')
                        torch.save(model, save_to_dir +'modelt1.pt')
#                         torch.save({
#                         'epoch': epoch,
#                         'model_state_dict': model.state_dict(),
#                         'optimizer_state_dict': optimizer.state_dict(),
#                         'loss': Loss
#                         }, save_to_dir +'modelt2.pt')

                
        ## Get the Layer Encodeing base on the layer number
    scheduler.step()
    print(scheduler.get_last_lr())
        

In [ ]:
mat = np.random.rand(4,4)
print(mat)
fft = np.fft.fft2(mat)
# fft = np.fft.fftshift(fft)
print(fft.real)
print(fft.imag)
print(fft[1:,1:] - np.conj(np.rot90(fft[1:,1:],2)))

# fft = np.fft.fftshift(fft)

# print(fft.real)
# print(fft.imag)

# fft = np.fft.fftshift(fft)
# pred = np.fft.ifft(fft,n=6)
# print(pred)

In [ ]:
v = np.arange(1)
v
np.max(v[0:-1])